In [1]:
import os
from pyspark.sql import SparkSession
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'
spark = SparkSession.builder.appName("Sensor_data")\
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2").master("local[2]").getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nithishra45hedu/.ivy2/cache
The jars for the packages stored in: /home/nithishra45hedu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-933c7f6d-c62f-4553-b33b-8cc106043fb9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 942ms :: artifacts dl 76ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;

In [2]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers","master:9092").option("subscribe","nitSensor").load()

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
sensorSchema = StructType([
    StructField("timestamp", TimestampType(),True),
    StructField("sensor_id", StringType(),True),
    StructField("value",DoubleType(),True)
])

In [5]:
sensorData = df.select(from_csv(df.value.cast("string"),"timestamp timestamp, sensor_id string, value double").alias("val")).select("val.*")

In [7]:
sensorData.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- sensor_id: string (nullable = true)
 |-- value: double (nullable = true)



In [8]:
sensorRes = sensorData.withWatermark("timestamp","1 minute")\
.groupBy(window(sensorData.timestamp,"1 minute", slideDuration="1 minute"),sensorData.sensor_id)\
.agg(avg(sensorData.value).alias("Avg"))

In [13]:
sensorRes.select("window.start","window.end","sensor_id","Avg").printSchema()
# sensorRes.printSchema()

root
 |-- start: timestamp (nullable = true)
 |-- end: timestamp (nullable = true)
 |-- sensor_id: string (nullable = true)
 |-- Avg: double (nullable = true)



In [14]:
sensorRes.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- sensor_id: string (nullable = true)
 |-- Avg: double (nullable = true)



In [10]:
spark.conf.set("spark.sql.streaming.checkpointLocation", "sensorchk1")

In [12]:
sensorRes.select("window.start","window.end","sensor_id","Avg").writeStream.format("console").outputMode("update").start().awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---+---------+----------+
|start|end|sensor_id|runningAvg|
+-----+---+---------+----------+
+-----+---+---------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-------------------+---------+------------------+
|              start|                end|sensor_id|        runningAvg|
+-------------------+-------------------+---------+------------------+
|2025-06-19 00:49:00|2025-06-19 00:50:00| sensor_C|              55.7|
|2025-06-19 00:48:00|2025-06-19 00:49:00| sensor_A|             72.45|
|2025-06-19 00:45:00|2025-06-19 00:46:00| sensor_B|             50.62|
|2025-06-19 00:48:00|2025-06-19 00:49:00| sensor_B|             37.22|
|2025-06-19 00:45:00|2025-06-19 00:46:00| sensor_C|             50.32|
|2025-06-19 00:49:00|2025-06-19 00:50:00| sensor_B|             61.43|
|2025-06-19 00:46:00|2025-06-19 00:47:00| sensor_B|              37.1|
|2025-06-19 00:47:00|2025-06-19 00:48:00| sensor_C|56.989999999999995|
|2025-06-19 00:46:00|2025-06-19 00:47:00| sensor_C|             59.22|
+-------------------+-------------------+---------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+---+---------+----------+
|start|end|sensor_id|runningAvg|
+-----+---+---------+----------+
+-----+---+---------+----------+



KeyboardInterrupt: 